# IMLAB pytorch study day1 (Training classifier)

### 패키지 import

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

### 데이터셋 불러오기

In [13]:
# transform은 [0,1] data를 [-1,1] range로 바꿔주는 과정입니다.
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# train set 불러오기 
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

# tensorflow와 다르게 placeholder 지정해주는 것이 아니라 (tensorflow도 물론 queue를 이용해 바로 학습할 수도 있지만)
# DataLoader class를 호출하여 바로 dataset을 불러옵니다.

trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)
"""
    Data loader. Combines a dataset and a sampler, and provides
    single- or multi-process iterators over the dataset.

    Arguments:
        dataset (Dataset): dataset from which to load the data.
        batch_size (int, optional): how many samples per batch to load
            (default: 1).
        shuffle (bool, optional): set to ``True`` to have the data reshuffled
            at every epoch (default: False).
        sampler (Sampler, optional): defines the strategy to draw samples from
            the dataset. If specified, ``shuffle`` must be False.
        batch_sampler (Sampler, optional): like sampler, but returns a batch of
            indices at a time. Mutually exclusive with batch_size, shuffle,
            sampler, and drop_last.
        num_workers (int, optional): how many subprocesses to use for data
            loading. 0 means that the data will be loaded in the main process
            (default: 0)
        collate_fn (callable, optional): merges a list of samples to form a mini-batch.
        pin_memory (bool, optional): If ``True``, the data loader will copy tensors
            into CUDA pinned memory before returning them.
        drop_last (bool, optional): set to ``True`` to drop the last incomplete batch,
            if the dataset size is not divisible by the batch size. If False and
            the size of dataset is not divisible by the batch size, then the last batch
            will be smaller. (default: False)
"""

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [5]:
trainset.train_data.shape

(50000, 32, 32, 3)

In [9]:
dataiter = iter(trainloader)
images, labels = dataiter.next()

In [22]:
images.view?

### Neural network 구조

In [25]:
# torch는 필수적으로 사용하는 패키지들이 정해져 있습니다. torch.nn(nn), torch.autograd.Variable (Variable), torch.nn.functional (F)
# 자주 사용하기 때문에 torch.nn torch.autograd.Variable 이렇게 매번 쓰기보다 직접 선언해놓는것이 편합니다.

# autograd package에 Variable class를 import 해줍니다.
# 흔히 텐서플로우의 tf.Variable 또는 tf.get_variable이라고 생각하시면 됩니다.
from torch.autograd import Variable

# convolution, dense layer 등 layer class들이 포함되어 있습니다.
# loss 계산에 필요한 loss class들도 nn package에 포함되어 있습니다.
# nn.Module class를 보통 상속받아 원하는 네트워크 구조를 만듭니다.
import torch.nn as nn

# non linear 함수 등 다양한 함수를 포함하고있는 class 입니다.
import torch.nn.functional as F


In [26]:
# 먼저 network 구조를 만듭니다. nn.Module class를 상속받아 class를 새로 생성합니다. tensorflow에서 graph를 build 하는 것과 유사합니다.
class Net(nn.Module):
    
    # 여기서 본인이 원하는 network 구조를 만들어줍니다.
    # 간단한 convolutional network를 예시로 보여주고 있습니다. layer를 쌓는 형태로 선언해주면 됩니다. 
    # 단, 여기서(__init__)는 이어주지 않고 사용할 layer를 선언만 해줍니다. 실제로 계산은 forward 함수에 선언합니다.
    def __init__(self):
        
        # 부모 class의 constructor를 불러옵니다.
        super(Net, self).__init__()
        
        # Conv2d의 parameter들은 차례로 [input_channel, output_channel, kernel_size] 입니다.
        self.conv1 = nn.Conv2d(3,6,5)
        # Maxpool의 parameter들은 [kernel_size, stride] 입니다.
        self.pool = nn.MaxPool2d(2,2)
        # pool에서 나온 channel갯수가 6일것이므로 input channel에 6을 넣는식입니다.
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        # 마지막 class 갯수만큼 확률값을 내뱉도록 network구조를 완성합니다.
        self.fc3 = nn.Linear(84, 10)

    
    # __init__에서 선언한 layer들을 이어줄 차례입니다.
    # nn.Module의 forward함수를 override하는것이기때문에 무조건 forward 함수로 선언합니다.
    def forward(self, x):
        # input으로 data batch x를 받습니다.
        # tensorflow와 유사하게 쭉 따라가면서 layer를 쌓으면 됩니다.
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        # Tensor.view는 tensorflow의 reshape 함수와 동일합니다.
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        # logit을 return 해줍니다.
        return x

    
# network 구조가 다 완성되면 instance를 생성합니다.
net = Net()

### Loss function 선언하기

In [31]:
# tensorflow의 tf.train.000optimizer 처럼 optimizer들이 들어있는 package입니다.
import torch.optim as optim

# nn package에서 classification을 위한 Cross-Entropy loss를 가져옵니다.
criterion = nn.CrossEntropyLoss()

# optimizer를 선언합니다.
# 위에서 만든 network 구조에 있는 parameter 전부를 optimizer에 parameter로 넣어줍니다.
# momentum은 http://aikorea.org/cs231n/neural-networks-3/#sgd 참고
# weight decay도 parameter로 바로 넣어줄 수 있습니다.
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### Training 하기

In [32]:
for epoch in range(3):
    
    # 2000 step마다의 loss 평균을 찍어주기 위해 running_loss를 선언합니다. tensorflow reduce_mean 함수 같은건 없습니다.
    running_loss = 0.0
    
    # 위에서 선언한 trainloader를 iteration 합니다.
    for i, data in enumerate(trainloader, 0):
        # batch만큼의 data를 가져옵니다.
        inputs, labels = data
        
        # Variable로 감싸줍니다.
        inputs, labels = Variable(inputs), Variable(labels)
        
        # gradient를 매 batch step마다 0으로 초기화시켜주는 작업입니다.
        optimizer.zero_grad()
        
        # batch data를 network에 넣어 forward 해줍니다.
        outputs = net(inputs)

        # batch loss 계산합니다.
        # logit을 바로 넣어줍니다. softmax function은 아마도 criterion에서 알아서 계산해주는듯합니다.
        loss = criterion(outputs, labels)
        
        # backpropagation을 위한 과정입니다.
        loss.backward()
        optimizer.step()
        
        # logging
        # network를 지나는 variable들 안에있는 값을 들고오려면 .data를 해야합니다.
        running_loss += loss.data[0]
        if i % 2000 == 1999:
            print "[%d, %d] loss: %.4f" % (epoch+1, i+1, running_loss/2000)
            running_loss = 0.0
            
print "Training Done"

[1, 2000] loss: 1.8591
[1, 4000] loss: 1.7950
[1, 6000] loss: 1.7734
[1, 8000] loss: 1.7807
[1, 10000] loss: 1.7546
[1, 12000] loss: 1.7433
[2, 2000] loss: 1.7322
[2, 4000] loss: 1.7144
[2, 6000] loss: 1.6955
[2, 8000] loss: 1.6786
[2, 10000] loss: 1.7093
[2, 12000] loss: 1.6862
[3, 2000] loss: 1.6621
[3, 4000] loss: 1.6541
[3, 6000] loss: 1.6496
[3, 8000] loss: 1.6608
[3, 10000] loss: 1.6346
[3, 12000] loss: 1.6364
Training Done


### Test 하기

In [33]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# Ground Truth를 뽑습니다.
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

('GroundTruth: ', '  cat  ship  ship plane')


In [34]:
outputs = net(Variable(images))

In [35]:
outputs

Variable containing:
-1.0129 -1.4325  0.2734  1.6490  0.2815  0.7594  1.6694 -0.3623 -0.8485 -0.6891
 2.8998  1.9432 -0.0190 -1.2469 -0.4915 -2.3794 -2.7576 -1.3390  3.0353  1.6208
 2.4572  1.9511 -0.4096 -1.7739 -0.6269 -2.7192 -2.1823 -1.4018  3.2863  2.3461
 2.4308  2.0453 -0.1739 -1.0787 -0.6585 -1.9802 -2.3457 -1.4036  2.6141  1.5711
[torch.FloatTensor of size 4x10]

In [40]:
# max value와 argmax를 동시에 뱉어줍니다. axis=1로 max한 값입니다.
_, predictions = torch.max(outputs.data, 1)

In [53]:
correct = 0
total = 0
for data in testloader:
    images, labels = data
    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 38 %


In [55]:
# [0,1] 마스킹 list가 나옵니다. 맞으면 1, 틀리면 0입니다.
(predictions == labels).sum()

0

### GPU 사용

In [56]:
# 어떻게 하는지 잘 모르겠습니다. cuda가 없어서 에러가 났을수도.
net.cuda()
inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())

AssertionError: Torch not compiled with CUDA enabled